In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("APLHAVANTAGEAPIKEY")

print(api_key)

3C52DWMLWFR27QAR


In [2]:
import requests

url = "https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=IBM&apikey=" + api_key
response = requests.get(url)
print(response.status_code)
data = response.json()

# Display the latest 5 days of daily prices in a readable format
time_series = data.get("Time Series (Daily)", {})
for date in sorted(time_series.keys(), reverse=True)[:5]:
    print(f"Date: {date}")
    for key, value in time_series[date].items():
        print(f"  {key}: {value}")
    print("-" * 30)


200
Date: 2025-06-04
  1. open: 264.9000
  2. high: 267.0000
  3. low: 264.7900
  4. close: 265.5200
  5. volume: 2588741
------------------------------
Date: 2025-06-03
  1. open: 263.3500
  2. high: 265.5600
  3. low: 262.5800
  4. close: 265.2000
  5. volume: 2494922
------------------------------
Date: 2025-06-02
  1. open: 257.8500
  2. high: 263.9760
  3. low: 257.2200
  4. close: 263.9000
  5. volume: 2831881
------------------------------
Date: 2025-05-30
  1. open: 258.7500
  2. high: 260.1200
  3. low: 257.1000
  4. close: 259.0600
  5. volume: 9668923
------------------------------
Date: 2025-05-29
  1. open: 260.7500
  2. high: 261.1300
  3. low: 256.7700
  4. close: 258.6900
  5. volume: 2295228
------------------------------


In [3]:
# I want to store the data in a SQL database. Columns should be: Date, Open, High, Low, Close, Volume.


import sqlite3
def create_table():
    conn = sqlite3.connect('stock_data.db')
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS daily_prices (
            Date TEXT PRIMARY KEY,
            Open REAL,
            High REAL,
            Low REAL,
            Close REAL,
            Volume INTEGER
        )
    ''')
    conn.commit()
    conn.close()    

def insert_data(date, open_price, high_price, low_price, close_price, volume):
    conn = sqlite3.connect('stock_data.db')
    cursor = conn.cursor()
    cursor.execute('''
        INSERT OR REPLACE INTO daily_prices (Date, Open, High, Low, Close, Volume)
        VALUES (?, ?, ?, ?, ?, ?)
    ''', (date, open_price, high_price, low_price, close_price, volume))
    conn.commit()
    conn.close()

create_table()

# Insert the data into the database
for date, prices in time_series.items():
    insert_data(
        date,
        float(prices["1. open"]),
        float(prices["2. high"]),
        float(prices["3. low"]),
        float(prices["4. close"]),
        int(prices["5. volume"])
    )

# Query the database to verify the data was inserted correctly
def query_data():
    conn = sqlite3.connect('stock_data.db')
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM daily_prices ORDER BY Date DESC LIMIT 5')
    rows = cursor.fetchall()
    conn.close()
    return rows

# Print the last 5 entries from the database
last_entries = query_data()
for entry in last_entries:
    print(f"Date: {entry[0]}, Open: {entry[1]}, High: {entry[2]}, Low: {entry[3]}, Close: {entry[4]}, Volume: {entry[5]}")
# The code above creates a SQLite database, stores daily stock prices, and retrieves the last 5 entries.
# Ensure the database and table are created before inserting data.




Date: 2025-06-04, Open: 264.9, High: 267.0, Low: 264.79, Close: 265.52, Volume: 2588741
Date: 2025-06-03, Open: 263.35, High: 265.56, Low: 262.58, Close: 265.2, Volume: 2494922
Date: 2025-06-02, Open: 257.85, High: 263.976, Low: 257.22, Close: 263.9, Volume: 2831881
Date: 2025-05-30, Open: 258.75, High: 260.12, Low: 257.1, Close: 259.06, Volume: 9668923
Date: 2025-05-29, Open: 260.75, High: 261.13, Low: 256.77, Close: 258.69, Volume: 2295228


In [4]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

url = "https://u.today/search/node?keys=bitcoin"
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Extract article titles, dates, authors, and links
articles = soup.find_all('div', class_='news__item')
data = []
seen = set()
for article in articles:
    # Title
    title_tag = article.find('div', class_='news__item-title')
    title = title_tag.get_text(strip=True) if title_tag else 'No Title'
    # Date
    date_tag = article.find('div', class_='humble')
    date = date_tag.get_text(strip=True) if date_tag else 'No Date'
    # Author
    author_tag = article.find('a', class_='humble humble--author')
    author = author_tag.get_text(strip=True) if author_tag else 'Unknown'
    # Link
    link_tag = article.find('a', class_='news__item-body')
    link = link_tag['href'] if link_tag and link_tag.has_attr('href') else 'No Link'

    # Use (title, date, author, link) as a unique identifier
    identifier = (title, date, author, link)
    if identifier not in seen:
        data.append({
            'Title': title,
            'Date': date,
            'Author': author,
            'Link': link
        })
        seen.add(identifier)

# Create a DataFrame from the scraped data
df = pd.DataFrame(data)
df

,Title,Date,Author,Link
0,'Rich Dad Poor Dad' Author: Bitcoin to $1 Mill...,"Jun 5, 2025 - 10:06",Yuri Molchan,https://u.today/rich-dad-poor-dad-author-bitco...
1,Bitcoin Block #899826 Mined by Solo Miner: Her...,"Jun 5, 2025 - 8:55",Arman Shirinyan,https://u.today/bitcoin-block-899826-mined-by-...
2,NBA Legend Scottie Pippen Issues Major Bitcoin...,"Jun 5, 2025 - 8:04",Yuri Molchan,https://u.today/nba-legend-scottie-pippen-issu...
3,"Bitcoin (BTC) Signals Mini-Death Cross, This S...","Jun 5, 2025 - 0:01",Arman Shirinyan,https://u.today/bitcoin-btc-signals-mini-death...
4,'Buy Bitcoin': Musk's $5 Trillion Debt Warning...,"Jun 4, 2025 - 20:14",Alex Dovbnya,https://u.today/buy-bitcoin-musks-5-trillion-d...
5,Only 320 Billion Shiba Inu Transacted in 24 Ho...,"Jun 4, 2025 - 16:41",Valeria Blokhina,https://u.today/only-320-billion-shiba-inu-tra...
6,Breaking: JPMorgan Dives Deeper Into Crypto wi...,"Jun 4, 2025 - 15:41",Alex Dovbnya,https://u.today/breaking-jpmorgan-dives-deeper...
7,Shockingly Bullish Bitcoin Price Prediction Ma...,"Jun 4, 2025 - 15:03",Alex Dovbnya,https://u.today/shockingly-bullish-bitcoin-pri...
8,"Bitcoin to Hit $115,000 By Week's End, Predict...","Jun 4, 2025 - 14:51",Yuri Molchan,https://u.today/bitcoin-to-hit-115000-by-weeks...
9,Who Really Owns Bitcoin ETFs Now? Top Holders ...,"Jun 4, 2025 - 14:16",Gamza Khanzadaev,https://u.today/who-really-owns-bitcoin-etfs-n...


In [ ]:
# Save the DataFrame to a CSV file
df.to_csv('articles.csv', index=False)

## Streamlit

In [ ]:
import streamlit as st

st.title("Hello World")
